<a href="https://colab.research.google.com/github/reskyagus21/Fine-tuning/blob/main/Unsloth_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("json", data_files="/content/drive/MyDrive/Dataset/Dataset.jsonl", split="train")
df = pd.DataFrame(dataset)
df

Generating train split: 0 examples [00:00, ? examples/s]

,instruction,input,output
0,Bagaimana peran Bajau dalam perkembangan TI In...,,Bajau memiliki peran penting dalam mendorong p...
1,Apa filosofi awal yang diusung oleh Bajau?,,Filosofi Bajau sejak awal adalah menyediakan s...
2,Apa saja pencapaian Bajau selama 25 tahun?,,"Dalam 25 tahun, Bajau telah melahirkan berbaga..."
3,Apakah Bajau mempengaruhi kebijakan TI nasional?,,"Secara tidak langsung, gerakan open source yan..."
4,Bagaimana Bajau beradaptasi dengan perubahan t...,,Bajau terus berinovasi dengan mengadopsi tekno...
...,...,...,...
945,Bagaimana Bajau menjaga relevansi selama 2 dek...,,"Dengan selalu belajar, beradaptasi, dan memban..."
946,Apa kontribusi Bajau untuk industri TI Indonesia?,,"Bajau turut mendorong adopsi open source, pela..."
947,Bagaimana Bajau mengembangkan inovasi dari pen...,,Dengan mengevaluasi kesuksesan dan kegagalan u...
948,Mengapa usia 25 tahun penting dalam dunia bisnis?,,"Karena menunjukkan kematangan organisasi, stab..."


In [5]:
datakosong = df.isnull().sum()
print(datakosong)

instruction    0
input          0
output         0
dtype: int64


In [ ]:
%%capture
!pip install unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2000
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
    )


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.1: Fast Mistral patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0.2,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.2.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.8.1 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
model.print_trainable_parameters()

trainable params: 167,772,160 || all params: 7,409,504,256 || trainable%: 2.2643


In [ ]:
import json

input_path = "/content/drive/MyDrive/Dataset/Dataset.jsonl"
output_path = "/content/drive/MyDrive/Dataset/converted_conversations.jsonl"

with open(input_path, 'r', encoding='utf-8') as infile, open(output_path, 'w', encoding='utf-8') as outfile:
    for line_number, line in enumerate(infile, start=1):
        line = line.strip()
        if not line:
            continue
        try:
            data = json.loads(line)
            instruction = data.get("instruction", "").strip()
            output = data.get("output", "").strip()

            conv = {
                "messages": [
                    {"role": "system", "content": "Chatbot PT Bajau Escorindo"},
                    {"role": "user", "content": instruction},
                    {"role": "assistant", "content": output}
                ]
            }

            json.dump(conv, outfile, ensure_ascii=False)
            outfile.write('\n')
        except json.JSONDecodeError as e:
            print(f"[!] Error decoding JSON di baris {line_number}: {e}")


In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/drive/MyDrive/Dataset/converted_conversations.jsonl", split="train")

def formatting(example):
    return {"text": tokenizer.apply_chat_template(example["messages"], tokenize=False)}

dataset = dataset.map(formatting)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/950 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc = 2,
    packing = False,

    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "Bajau-Mistral-7b-4bit",
        report_to = "none",
    ),
)


Unsloth: Tokenizing ["text"]:   0%|          | 0/950 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
dataset = load_dataset("json", data_files="/content/drive/MyDrive/Dataset/converted_conversations.jsonl", split="train")
df = pd.DataFrame(dataset)
df


,messages
0,"[{'role': 'system', 'content': 'Chatbot PT Baj..."
1,"[{'role': 'system', 'content': 'Chatbot PT Baj..."
2,"[{'role': 'system', 'content': 'Chatbot PT Baj..."
3,"[{'role': 'system', 'content': 'Chatbot PT Baj..."
4,"[{'role': 'system', 'content': 'Chatbot PT Baj..."
...,...
945,"[{'role': 'system', 'content': 'Chatbot PT Baj..."
946,"[{'role': 'system', 'content': 'Chatbot PT Baj..."
947,"[{'role': 'system', 'content': 'Chatbot PT Baj..."
948,"[{'role': 'system', 'content': 'Chatbot PT Baj..."


In [ ]:
tokenizer.decode(trainer.train_dataset[1]["input_ids"])

'<s><s> [INST] Chatbot PT Bajau Escorindo\n\nApa filosofi awal yang diusung oleh Bajau? [/INST] Filosofi Bajau sejak awal adalah menyediakan solusi TI yang inklusif dan dapat diakses oleh berbagai kalangan, dengan semangat kolaborasi komunitas.</s>'

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 950 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 167,772,160 of 7,409,504,256 (2.26% trained)


Step,Training Loss
1,5.062100
2,4.970200
3,4.872300
4,4.164400
5,3.818800
6,2.630700
7,2.180300
8,1.776900
9,1.768800
10,1.586100


In [ ]:
from transformers import AutoTokenizer

prompt = """### User:
Siapa pendiri PT Bajau ?

### Assistant:
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
    input_ids=inputs["input_ids"],
    max_new_tokens=128,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


### User:
Siapa pendiri PT Bajau ?

### Assistant:
Mang Sugeng, Dwi Wahyu, dan Ida Bagus.


In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 831.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

base_model = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"
lora_model = "/content/drive/MyDrive/Dataset/Bajau-Mistral-7b-4bit"  # Contoh, ganti sesuai model kamu

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model,use_fast=False)

# Load base model
base = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load LoRA adapter tanpa merge
model = PeftModel.from_pretrained(base, lora_model)

# Evaluation mode
model.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.2, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
      

In [ ]:
prompt = [{"role": "user", "content": "Apa arti nama bajau?"}]
inputs = tokenizer.apply_chat_template(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=128,
        do_sample=False
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
!cp -r /content/outputs/Bajau-Mistral-7b-4bit/ /content/drive/MyDrive/Dataset/